# AquaAdvisor Demo Analysis

This notebook demonstrates the core functionality of the AquaAdvisor system for water stress detection and irrigation recommendations.

## Overview

In this notebook, we'll walk through the complete analysis pipeline:
1. Loading sample satellite data
2. Calculating NDVI (Normalized Difference Vegetation Index)
3. Detecting water stress zones
4. Generating irrigation recommendations
5. Visualizing results

This demo uses synthetic data to illustrate the system's capabilities without requiring API access.

## 1. Setup

First, let's install the required packages and set up our environment.

In [ ]:
# Install required packages
!pip install numpy matplotlib scipy pillow

In [ ]:
import sys
sys.path.append('../backend')

## 2. Import Modules

Let's import all the necessary modules for our analysis.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from ndvi_processor import NDVIProcessor
from stress_analyzer import StressAnalyzer
from recommendation_engine import RecommendationEngine
from weather_service import WeatherService
import sys
import os

# Set up plotting
%matplotlib inline
plt.style.use('seaborn-v0_8')

# Add backend to path if needed
backend_path = os.path.join(os.getcwd(), '..', 'backend')
if backend_path not in sys.path:
    sys.path.append(backend_path)

## 3. Load Sample Data

We'll generate synthetic satellite data to demonstrate the analysis pipeline.

In [ ]:
# Generate synthetic data
np.random.seed(42)  # For reproducible results

# Create a 100x100 pixel image
height, width = 100, 100

# Generate red band (100-180)
red = np.random.randint(100, 181, size=(height, width)).astype(np.float32)

# Generate NIR band (180-255)
nir = np.random.randint(180, 256, size=(height, width)).astype(np.float32)

# Visualize the bands
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

im1 = axes[0].imshow(red, cmap='Reds')
axes[0].set_title('Red Band')
plt.colorbar(im1, ax=axes[0])

im2 = axes[1].imshow(nir, cmap='Greens')
axes[1].set_title('NIR Band')
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

print(f"Red band shape: {red.shape}, range: {red.min():.1f}-{red.max():.1f}")
print(f"NIR band shape: {nir.shape}, range: {nir.min():.1f}-{nir.max():.1f}")

## 4. Calculate NDVI

Now we'll calculate the NDVI using the formula: NDVI = (NIR - Red) / (NIR + Red)

In [ ]:
# Calculate NDVI
ndvi_processor = NDVIProcessor()
ndvi = ndvi_processor.calculate_ndvi(red, nir)
smoothed_ndvi = ndvi_processor.smooth_ndvi(ndvi)

# Calculate statistics
stats = ndvi_processor.calculate_statistics(smoothed_ndvi)

# Visualize NDVI
plt.figure(figsize=(10, 8))
im = plt.imshow(smoothed_ndvi, cmap='RdYlGn', vmin=-1, vmax=1)
plt.colorbar(im, label='NDVI')
plt.title('Normalized Difference Vegetation Index (NDVI)')
plt.xlabel('Pixel')
plt.ylabel('Pixel')
plt.show()

print("NDVI Statistics:")
for key, value in stats.items():
    print(f"  {key}: {value:.4f}")

## 5. Simulate Weather Data

We'll simulate weather data to demonstrate how it affects our analysis.

In [ ]:
# Simulate weather data
weather_service = WeatherService()

# For demo purposes, we'll use sample data
weather = weather_service.get_current_weather(36.6, -120.375)
print("Current Weather Conditions:")
for key, value in weather.items():
    print(f"  {key}: {value}")

# Calculate ET0 (reference evapotranspiration)
et0 = weather_service.calculate_et0(weather)
print(f"\nReference Evapotranspiration (ET0): {et0:.2f} mm/day")

# Estimate rainfall
rainfall = weather_service.estimate_rainfall(36.6, -120.375, days=7)
print(f"Estimated Rainfall (7 days): {rainfall:.2f} mm")

# Calculate water deficit
deficit = weather_service.assess_water_deficit(36.6, -120.375)
print("\nWater Deficit Assessment:")
for key, value in deficit.items():
    print(f"  {key}: {value}")

## 6. Detect Stress Zones

Now we'll classify the field into different stress zones based on NDVI values.

In [ ]:
# Detect stress zones
stress_analyzer = StressAnalyzer()
stress_zones = stress_analyzer.detect_stress_zones(smoothed_ndvi, weather)

# Calculate zone statistics
zone_stats = stress_analyzer.calculate_zone_statistics(stress_zones, smoothed_ndvi)

# Analyze quadrants
quadrant_analysis = stress_analyzer.analyze_quadrants(smoothed_ndvi)

# Visualize stress zones
stress_map = stress_analyzer.create_stress_map(stress_zones)

# Display stress zone map
plt.figure(figsize=(10, 8))
# Create RGB image from zones
colors = np.array([
    [1.0, 0.0, 0.0],  # Red - Critical stress
    [1.0, 0.65, 0.0], # Orange - High stress
    [1.0, 1.0, 0.0],  # Yellow - Moderate stress
    [0.0, 1.0, 0.0]   # Green - Healthy
])
rgb_image = colors[stress_zones]
plt.imshow(rgb_image)
plt.title('Water Stress Zones')
plt.xlabel('Pixel')
plt.ylabel('Pixel')

# Create custom legend
import matplotlib.patches as mpatches
legend_patches = [
    mpatches.Patch(color='red', label='Critical Stress'),
    mpatches.Patch(color='orange', label='High Stress'),
    mpatches.Patch(color='yellow', label='Moderate Stress'),
    mpatches.Patch(color='green', label='Healthy')
]
plt.legend(handles=legend_patches, loc='upper right')

plt.show()

print("Stress Zone Distribution:")
for zone, data in zone_stats.items():
    print(f"  {zone}: {data['percentage']:.1f}% (mean NDVI: {data['mean_ndvi']:.3f})")

print("\nQuadrant Analysis:")
for quadrant, data in quadrant_analysis.items():
    print(f"  {quadrant}: mean NDVI = {data['mean_ndvi']:.3f}, stressed = {data['stressed_percentage']:.1f}%")

## 7. Generate Recommendations

Based on our analysis, we'll generate irrigation recommendations.

In [ ]:
# Generate recommendations
recommendation_engine = RecommendationEngine()
recommendations = recommendation_engine.generate_recommendations(
    zone_stats, quadrant_analysis, deficit, stats['mean']
)

water_savings = recommendation_engine.calculate_water_savings(zone_stats)

print("Irrigation Recommendations:")
print("=" * 50)
for i, rec in enumerate(recommendations, 1):
    print(f"{i}. Priority: {rec['priority']} ({rec['urgency']})")
    print(f"   Zone: {rec['zone']}")
    print(f"   Action: {rec['action']}")
    print(f"   Reason: {rec['reason']}")
    print(f"   Water Amount: {rec['water_amount']}")
    print(f"   Timing: {rec['timing']}")
    print(f"   Cost Impact: {rec['cost_impact']}")
    print()

print("Water Efficiency:")
print(f"  Potential Savings: {water_savings['savings_percentage']:.1f}%")
print(f"  Water Savings: {water_savings['savings_mm']:.1f} mm per season")
print(f"  Explanation: {water_savings['explanation']}")

## 8. Comparison: Uniform vs Targeted Irrigation

Let's compare the efficiency of uniform irrigation versus our targeted approach.

In [ ]:
# Simulate uniform irrigation approach
uniform_water_use = 100  # mm (example)
targeted_water_use = uniform_water_use - water_savings['savings_mm']

print("Irrigation Efficiency Comparison:")
print("=" * 40)
print(f"Uniform Irrigation: {uniform_water_use:.1f} mm")
print(f"Targeted Irrigation: {targeted_water_use:.1f} mm")
print(f"Water Savings: {water_savings['savings_mm']:.1f} mm ({water_savings['savings_percentage']:.1f}%)")

# Visualize comparison
approaches = ['Uniform\nIrrigation', 'Targeted\nIrrigation']
water_usage = [uniform_water_use, targeted_water_use]

plt.figure(figsize=(8, 6))
bars = plt.bar(approaches, water_usage, color=['#1f77b4', '#2ca02c'])
plt.ylabel('Water Usage (mm)')
plt.title('Water Usage Comparison')
plt.ylim(0, max(water_usage) * 1.1)

# Add value labels on bars
for bar, value in zip(bars, water_usage):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 1,
             f'{value:.1f} mm', ha='center', va='bottom')

plt.show()

## 9. Edge Cases

Let's examine how our system handles various edge cases.

In [ ]:
# Test with different scenarios

# Scenario 1: Drought conditions (low NDVI)
print("Scenario 1: Drought Conditions")
print("-" * 30)
drought_ndvi = np.full((50, 50), 0.2)  # Very low NDVI
drought_zones = stress_analyzer.detect_stress_zones(drought_ndvi)
drought_stats = stress_analyzer.calculate_zone_statistics(drought_zones, drought_ndvi)
for zone, data in drought_stats.items():
    print(f"  {zone}: {data['percentage']:.1f}%")

print()

# Scenario 2: Healthy field (high NDVI)
print("Scenario 2: Healthy Field")
print("-" * 25)
healthy_ndvi = np.full((50, 50), 0.8)  # High NDVI
healthy_zones = stress_analyzer.detect_stress_zones(healthy_ndvi)
healthy_stats = stress_analyzer.calculate_zone_statistics(healthy_zones, healthy_ndvi)
for zone, data in healthy_stats.items():
    print(f"  {zone}: {data['percentage']:.1f}%")

print()

# Scenario 3: Uneven stress distribution
print("Scenario 3: Uneven Stress Distribution")
print("-" * 40)
# Create a field with different stress levels in different areas
uneven_ndvi = np.zeros((100, 100))
uneven_ndvi[:50, :50] = 0.2  # Critical stress (NW quadrant)
uneven_ndvi[:50, 50:] = 0.4  # High stress (NE quadrant)
uneven_ndvi[50:, :50] = 0.6  # Moderate stress (SW quadrant)
uneven_ndvi[50:, 50:] = 0.8  # Healthy (SE quadrant)
uneven_zones = stress_analyzer.detect_stress_zones(uneven_ndvi)
uneven_quadrants = stress_analyzer.analyze_quadrants(uneven_ndvi)
for quadrant, data in uneven_quadrants.items():
    print(f"  {quadrant}: mean NDVI = {data['mean_ndvi']:.1f}, stressed = {data['stressed_percentage']:.1f}%")

## 10. Performance Metrics

Let's evaluate the performance of our analysis pipeline.

In [ ]:
import time

# Measure processing time for different operations
print("Performance Metrics:")
print("=" * 20)

# NDVI calculation
start_time = time.time()
ndvi = ndvi_processor.calculate_ndvi(red, nir)
ndvi_time = time.time() - start_time
print(f"NDVI Calculation: {ndvi_time*1000:.2f} ms")

# Stress zone detection
start_time = time.time()
stress_zones = stress_analyzer.detect_stress_zones(ndvi)
stress_time = time.time() - start_time
print(f"Stress Zone Detection: {stress_time*1000:.2f} ms")

# Recommendation generation
start_time = time.time()
recommendations = recommendation_engine.generate_recommendations(
    zone_stats, quadrant_analysis, deficit, stats['mean']
)
rec_time = time.time() - start_time
print(f"Recommendation Generation: {rec_time*1000:.2f} ms")

# Total processing time
total_time = ndvi_time + stress_time + rec_time
print(f"\nTotal Processing Time: {total_time*1000:.2f} ms")
print(f"Processing Speed: {1/total_time:.2f} fields/second")

## 11. Validation

Let's validate our results to ensure they make sense.

In [ ]:
# Validate NDVI values
print("Validation Results:")
print("=" * 20)

print(f"NDVI Range Check: {np.min(ndvi):.3f} to {np.max(ndvi):.3f}")
print(f"Expected Range: -1.000 to 1.000")
ndvi_valid = np.min(ndvi) >= -1.0 and np.max(ndvi) <= 1.0
print(f"NDVI Range Valid: {ndvi_valid}")

print()

# Validate stress zone percentages sum to 100%
total_percentage = sum(data['percentage'] for data in zone_stats.values())
print(f"Zone Percentages Sum: {total_percentage:.2f}%")
print(f"Expected Sum: 100.00%")
percentage_valid = abs(total_percentage - 100.0) < 0.1
print(f"Percentage Sum Valid: {percentage_valid}")

print()

# Validate quadrant analysis covers entire field
total_quadrant_pixels = sum(
    len(quadrant_analysis[q]['mean_ndvi']) for q in quadrant_analysis
)
total_field_pixels = ndvi.size
print(f"Quadrant Analysis Coverage: {total_quadrant_pixels}/{total_field_pixels} pixels")
coverage_valid = total_quadrant_pixels == total_field_pixels
print(f"Coverage Valid: {coverage_valid}")

## 12. Conclusions

Based on our analysis, we can draw the following conclusions:

1. **NDVI Calculation**: Our implementation correctly calculates NDVI values within the expected range of -1 to 1.

2. **Stress Detection**: The system successfully classifies fields into different stress zones based on vegetation health.

3. **Recommendations**: The recommendation engine generates actionable advice based on the analysis results.

4. **Water Savings**: Targeted irrigation can significantly reduce water usage while maintaining crop health.

5. **Performance**: The analysis pipeline processes fields quickly, making it suitable for real-time applications.

## References

1. Rouse, J.W., Rhoden, R.H., Deering, D.W., & Schell, J.A. (1974). Monitoring vegetation systems in the Great Plains with ERTS. NASA Special Publication, 351, 309-317.

2. Tucker, C.J. (1979). Red and photographic infrared linear combinations for monitoring vegetation. Remote Sensing of Environment, 8(2), 127-150.

3. FAO. (1998). Crop evapotranspiration - Guidelines for computing crop water requirements. FAO Irrigation and drainage paper 56.

4. European Space Agency. (2023). Sentinel-2 User Handbook.

This demo demonstrates the core functionality of the AquaAdvisor system. In a production environment, the system would integrate with real satellite data and weather APIs to provide accurate, up-to-date recommendations for farmers.